In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [2]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [3]:
train['TotalLivingSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalLivingSF'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

train['TotalLivingSF'] = np.log(train['TotalLivingSF'])
test['TotalLivingSF'] = np.log(test['TotalLivingSF'])

In [4]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
TotalLivingSF    0.698827
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr

In [5]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold', 'TotalLivingSF']


In [6]:
test['KitchenQual'] = test['KitchenQual'].fillna("TA") 

train['BsmtQual'] = train['BsmtQual'].fillna("NB") 
test['BsmtQual'] = test['BsmtQual'].fillna("NB") 

In [12]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'KitchenQual',
          'BsmtQual']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'KitchenQual', 'BsmtQual']


In [8]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(0, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(0, inplace=True)

In [9]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
TotalLivingSF    0.741062
GrLivArea        0.730255
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
GarageYrBlt      0.349014
WoodDeckSF       0.343269
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
LotFrontage      0.179303
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF

In [13]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 49 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [15]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 29)
(1459, 29)
(1460, 146)
(1459, 146)


In [20]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=500, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.001482529853905129

In [17]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [18]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

207.04785102122622

In [19]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [21]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.16662449124621317, 'OverallQual'),
 (0.1283878996528766, 'TotalLivingSF'),
 (0.12627950656016615, 'GrLivArea'),
 (0.07219668936895167, 'YearBuilt'),
 (0.05581060785788545, 'TA'),
 (0.04240692127589239, 'TotalBsmtSF'),
 (0.038752794712574964, 'GarageCars'),
 (0.038637800673864915, 'GarageArea'),
 (0.03644863391993991, 'GarageYrBlt'),
 (0.029806389558051656, '1stFlrSF'),
 (0.027137904921169388, 'FullBath'),
 (0.017439314612533438, 'LotArea'),
 (0.014971477659747032, 'YearRemodAdd'),
 (0.014736752630482552, 'Fireplaces'),
 (0.013065236360686827, '2ndFlrSF'),
 (0.011816224798214843, 'BsmtFinSF1'),
 (0.009828747882833484, 'OverallCond'),
 (0.009673097944233785, 'TA'),
 (0.00885567964483147, 'N'),
 (0.00826433694768465, 'Ex'),
 (0.00787224773116285, 'Y'),
 (0.007600535430225367, 'BsmtUnfSF'),
 (0.006751619834907817, 'TotRmsAbvGrd'),
 (0.005737510710680814, 'PConc'),
 (0.005266096311681493, 'OpenPorchSF'),
 (0.005039634314491839, 'LotFrontage'),
 (0.004930366697579581, 'Ex'),
 (0.00456257

In [18]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=450, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [0.13157721 0.12080204 0.13062403 0.17242126 0.15596222 0.11296874
 0.13557137 0.11685503 0.12921487 0.13700726]
mean: 0.13430040425834927
std: 0.01708729678845549
count    10.000000
mean      0.134300
std       0.018012
min       0.112969
25%       0.122905
50%       0.131101
75%       0.136648
max       0.172421
dtype: float64


In [24]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
#     {'n_estimators': [100, 200, 300, 350, 400, 450], 'max_features': [16, 32, 64]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 300, 400, 500, 600], 
     'max_features': [16, 32, 48]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=3, n_jobs=-1)
grid_search.fit(train_full_prepared, train_y)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  1.3min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'bootstrap': [False], 'max_featur

In [25]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 16, 'n_estimators': 500}

In [26]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=16, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [27]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.13538126179008947 {'bootstrap': False, 'max_features': 16, 'n_estimators': 100}
0.13436687236466066 {'bootstrap': False, 'max_features': 16, 'n_estimators': 300}
0.13419928382205157 {'bootstrap': False, 'max_features': 16, 'n_estimators': 400}
0.1339020713018217 {'bootstrap': False, 'max_features': 16, 'n_estimators': 500}
0.13395132455628853 {'bootstrap': False, 'max_features': 16, 'n_estimators': 600}
0.13559436357296048 {'bootstrap': False, 'max_features': 32, 'n_estimators': 100}
0.13457846481287164 {'bootstrap': False, 'max_features': 32, 'n_estimators': 300}
0.13454650481560695 {'bootstrap': False, 'max_features': 32, 'n_estimators': 400}
0.13446052644462006 {'bootstrap': False, 'max_features': 32, 'n_estimators': 500}
0.1343324197477185 {'bootstrap': False, 'max_features': 32, 'n_estimators': 600}
0.13608010438353638 {'bootstrap': False, 'max_features': 48, 'n_estimators': 100}
0.13509098510533998 {'bootstrap': False, 'max_features': 48, 'n_estimators': 300}
0.1353644847616678

In [28]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.02808220e-03, 6.99801920e-03, 2.22141359e-02, 1.09058180e-01,
       9.60884533e-03, 4.78349859e-02, 2.63698341e-02, 7.31194908e-03,
       1.71749792e-02, 8.44106375e-03, 5.08621994e-02, 3.95313145e-02,
       1.61953006e-02, 8.69035083e-02, 2.34178804e-03, 3.18699967e-02,
       4.25382608e-03, 5.76428328e-03, 1.97595123e-03, 1.57847504e-02,
       2.11405642e-02, 3.95565625e-02, 5.07764465e-02, 4.22142768e-02,
       5.14998000e-03, 9.07848635e-03, 2.33258389e-03, 2.53357797e-03,
       9.02844713e-02, 8.65295692e-05, 7.47662274e-06, 2.60278775e-04,
       3.78234426e-04, 5.53211002e-04, 3.91360443e-04, 1.68673160e-03,
       1.50198151e-03, 3.48638521e-04, 1.26976422e-03, 5.59179495e-04,
       2.97101263e-04, 8.38425992e-04, 4.25971846e-05, 3.62261576e-04,
       7.75145231e-04, 8.27464343e-04, 1.67709101e-03, 1.99705187e-04,
       4.88073790e-04, 2.18367763e-04, 4.12141227e-04, 3.56259965e-04,
       2.15281750e-04, 9.91511036e-05, 1.83896877e-03, 2.18288151e-04,
      

In [29]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.10905817965434608, 'OverallQual'),
 (0.09028447134503662, 'TotalLivingSF'),
 (0.08690350831038976, 'GrLivArea'),
 (0.050862199438885855, 'TotalBsmtSF'),
 (0.05077644653539286, 'GarageCars'),
 (0.0478349859143661, 'YearBuilt'),
 (0.04634784736734507, 'TA'),
 (0.04221427679545243, 'GarageArea'),
 (0.03955656246308664, 'GarageYrBlt'),
 (0.03953131452128913, '1stFlrSF'),
 (0.03186999665920121, 'FullBath'),
 (0.02636983407881924, 'YearRemodAdd'),
 (0.023336861733826157, 'TA'),
 (0.02221413586690065, 'LotArea'),
 (0.02114056421189362, 'Fireplaces'),
 (0.017174979167215346, 'BsmtFinSF1'),
 (0.016195300582788646, '2ndFlrSF'),
 (0.015784750368157043, 'TotRmsAbvGrd'),
 (0.013473049368388768, 'Gd'),
 (0.012036802761499136, 'PConc'),
 (0.011348516578202957, 'Ex'),
 (0.009608845332484937, 'OverallCond'),
 (0.009078486353513583, 'OpenPorchSF'),
 (0.008586622142391281, 'Ex'),
 (0.008488663372839775, 'TA'),
 (0.008441063751289727, 'BsmtUnfSF'),
 (0.008178947850363634, 'Y'),
 (0.00801886804182953, 

In [30]:
# y_pred = forest_reg.predict(test_full_prepared)
y_pred = grid_search.predict(test_full_prepared)
y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [31]:
sub.to_csv("data/submission_script38_BsmtQual.csv", index=False)

In [28]:
sub.shape

(1459, 2)